In [ ]:
# Importing all the packages we are going to need.

import requests
from bs4 import BeautifulSoup
import re

import numpy as np
import pandas as pd

import string
import time

In [ ]:
def get_profile_links(letter):


    time.sleep(5)

    base_url = f'https://en.khl.ru/players/season/all/?letter={letter}'

    result = requests.get(base_url)
    soup = BeautifulSoup(result.text, 'html.parser')
    result = requests.get(base_url)
    result_pandas = pd.read_html(result.text)

    links = []
    for link in soup.find_all('a', class_='players-table__player'):
      links.append(link.get('href').strip())
    player_info = result_pandas[-1]

    profile_links = pd.DataFrame(links, columns=['Profile link'])

    profile_links = pd.concat([profile_links,player_info], axis = 1)
    #goalkeepers = profile_links[profile_links['Position'] == 'Position goaltender']
    profile_links = profile_links.loc[profile_links['Position']  != 'Position goaltender']
    #print(position)
    return profile_links
get_profile_links('F')

,Profile link,Name Player,Team,Position,Born,Age,Country,Contract to
0,/players/17910/,Fabricius Karl,Lev (Poprad),Position forward,Born 29 September 1982,Age 40,Country Sweden,Contract to
1,/players/4537/,Fadeyev Evgeny,Barys (Astana),Position defense,Born 9 July 1982,Age 40,Country Kazakhstan,Contract to
2,/players/20356/,Fahey Brian,Vityaz (Moscow Region),Position defense,Born 2 March 1981,Age 42,Country USA,Contract to
3,/players/22526/,Fairchild Cade,Dinamo (Riga),Position defense,Born 15 January 1989,Age 34,Country USA,Contract to
4,/players/16688/,Faizullin Danil,Admiral (Vladivostok),Position forward,Born 3 December 1993,Age 29,Country Russia,Contract to 30.04.2024
5,/players/15054/,Fakhrutdinov Denis,Neftekhimik (Nizhnekamsk),Position forward,Born 21 March 1990,Age 33,Country Russia,Contract to
6,/players/14825/,Fakhrutdinov Marat,Traktor (Chelyabinsk) Lada (Togliatti),Position forward,Born 27 November 1988,Age 34,Country Russia,Contract to
7,/players/20724/,Falkovsky Stepan,SKA (Saint Petersburg),Position defense,Born 18 December 1996,Age 26,Country Belarus,Contract to 30.04.2026
9,/players/20124/,Fantenberg Oscar,SKA (Saint Petersburg),Position defense,Born 7 October 1991,Age 31,Country Sweden,Contract to
12,/players/22746/,Fateyev Yegor,Traktor (Chelyabinsk),Position forward,Born 3 November 1997,Age 25,Country Russia,Contract to 30.04.2024


In [ ]:
def scrape_player_info(profile_link):
  base_url = f'https://en.khl.ru/{profile_link}'
  result = requests.get(base_url)
  soup = BeautifulSoup(result.text, 'html.parser')
  infos = []
  for info in soup.find_all('p', class_='roboto roboto-bold roboto-lg roboto-lg__lg-bigger color-black'):
      infos.append(info.text.strip())

  headers = []
  for header in soup.find_all('p', class_ = 'roboto roboto-normal roboto-md color-black roboto-md_lh-ll'):
    headers.append(header.text.strip())

  player_info = pd.DataFrame([infos], columns=[headers])
  name = soup.find('span', class_='frameCard-header__detail-titleItem').text
  position = soup.find('p',class_='frameCard-header__detail-local').text.strip()
  player_name = pd.DataFrame([{ 'URL': base_url, 'Player name': name, 'Position' : position} ])


  return player_name, player_info
scrape_player_info('players/5516/')

(                               URL        Player name    Position
 0  https://en.khl.ru/players/5516/  Vasily Koshechkin  goaltender,
             Born Age Country Height Weight Shoots
 0  27 March 1983  40  Russia    200    115   left)

In [ ]:
def scrape_player_data(profile_link):


    time.sleep(5)


    #base_url = f'https://en.khl.ru/{profile_link}'
    #result = requests.get(base_url)
    #soup = BeautifulSoup(result.text, 'html.parser')
    base_url = f'https://en.khl.ru/{profile_link}'
    result = requests.get(base_url)
    result_pandas = pd.read_html(result.text)


    if len(result_pandas) < 2:
      #player_season_stat = result_pandas[-1]
      player_khl_stat = result_pandas[-1]
      player_games_in_khl_stat = None

    else:
      player_khl_stat = result_pandas[-2]
      player_games_in_khl_stat = result_pandas[-1]
      player_games_in_khl_stat.columns = ['Date', 'Teams', 'Score',	'№',	'G',	'Assists',	'PTS',	'+/-',	'+',	'-',	'PIM',	'ESG',	'PPG',	'SHG',	'OTG',	'GWG',	'SDS',	'SOG',	'%SOG',	'FO',	'FOW',	'%FO',	'TOI',	'SFT',	'TIE',	'SFTE',	'TIPP',	'SFTPP',	'TISH',	'SFTSH',	'TIEN',	'SFTEN',	'HITS',	'BLS',	'FOA',	'TkA']



    player_khl_stat.columns = ['Tournament/Team', '№',	'GP',	'G',	'Assists',	'PTS',	'+/-',	'+',	'-',	'PIM',	'ESG',	'PPG',	'SHG',	'OTG',	'GWG',	'SDS',	'SOG',	'%SOG',	'S/G',	'FO',	'FOW',	'%FO',	'TOI/G',	'SFT/G',	'TIE/G',	'SFTE/G',	'TIPP/G',	'SFTPP/G',	'TISH/G',	'SFTSH/G',	'HITS',	'BLS',	'FOA',	'TkA']
    #
    #if ('Tournament / Team GP Games played W Wins L Losses SOP Shootouts played SOG Shots on goal GA Goals allowed Sv Saves %Sv Saves percentage GAA Goals against average G Goals Assists Assists SO Shutouts PIM Penalty in minutes TOI Time on ice' in player_khl_stat.columns):
     # player_khl_stat = player_khl_stat.rename(columns={'Tournament / Team GP Games played W Wins L Losses SOP Shootouts played SOG Shots on goal GA Goals allowed Sv Saves %Sv Saves percentage GAA Goals against average G Goals Assists Assists SO Shutouts PIM Penalty in minutes TOI Time on ice': 'Tournament/Team'})
    #player_khl_stat = player_match_stat.columns=['Tournament/Team','№',	'GP',	'G',	'Assists',	'PTS',	'+/-',	'+',	'-',	'PIM',	'ESG',	'PPG',	'SHG',	'OTG',	'GWG',	'SDS',	'SOG',	'%SOG',	'S/G',	'FO',	'FOW',	'%FO',	'TOI/G',	'SFT/G',	'TIE/G',	'SFTE/G',	'TIPP/G',	'SFTPP/G',	'TISH/G',	'SFTSH/G',	'HITS',	'BLS',	'FOA',	'TkA']
    return player_khl_stat#, player_games_in_khl_stat
scrape_player_data('players/34721/')


,Tournament/Team,№,GP,G,Assists,PTS,+/-,+,-,PIM,...,TIE/G,SFTE/G,TIPP/G,SFTPP/G,TISH/G,SFTSH/G,HITS,BLS,FOA,TkA
0,21/22 | playoffs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jokerit (Helsinki),-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-,-,-,-,-,-,0.0,0.0,0.0,NaN
2,20/21 | playoffs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jokerit (Helsinki),25,2.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,...,12:37,17,01:08,1,00:00,0,4.0,0.0,1.0,NaN
4,20/21 | regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Jokerit (Helsinki),25,50.0,8.0,18.0,26.0,-6.0,25.0,31.0,24.0,...,12:05,17.1,02:09,2.76,00:01,0.08,24.0,3.0,12.0,NaN
6,19/20 | playoffs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Jokerit (Helsinki),25,6.0,2.0,2.0,4.0,4.0,6.0,2.0,0.0,...,11:00,17,03:06,3.67,00:00,0,3.0,2.0,0.0,NaN
8,19/20 | regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Jokerit (Helsinki),25,56.0,6.0,23.0,29.0,4.0,33.0,29.0,16.0,...,12:27,18.11,02:43,3.21,00:02,0.07,22.0,13.0,15.0,NaN


In [ ]:
def fix_player_khl_stat(player_khl_stat):
  #Добавить колонку сезон
  player_khl_stat['Season'] = np.where(np.isnan(player_khl_stat['GP']), player_khl_stat['Tournament/Team'], np.NaN)
  #В каждой строке прописать название сезона
  player_khl_stat['Season'] = player_khl_stat['Season'].fillna(method='ffill')


  player_khl_stat = player_khl_stat[player_khl_stat['Tournament/Team'] != player_khl_stat['Season']]

  player_khl_stat =  player_khl_stat[['Season'] + [col for col in player_khl_stat.columns if col != 'Season']]


  player_khl_stat = player_khl_stat.reset_index(drop=True)

  return player_khl_stat
player_khl_stat = scrape_player_data('players/34721/')
print(player_khl_stat)
fix_player_khl_stat(player_khl_stat)

       Tournament/Team    №     GP     G  Assists   PTS  +/-     +     -  \
0     21/22 | playoffs  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
1   Jokerit (Helsinki)    -    0.0   0.0      0.0   0.0  0.0   0.0   0.0   
2     20/21 | playoffs  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
3   Jokerit (Helsinki)   25    2.0   0.0      0.0   0.0 -1.0   0.0   1.0   
4      20/21 | regular  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
5   Jokerit (Helsinki)   25   50.0   8.0     18.0  26.0 -6.0  25.0  31.0   
6     19/20 | playoffs  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
7   Jokerit (Helsinki)   25    6.0   2.0      2.0   4.0  4.0   6.0   2.0   
8      19/20 | regular  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
9   Jokerit (Helsinki)   25   56.0   6.0     23.0  29.0  4.0  33.0  29.0   
10         KHL Summary  NaN    NaN   NaN      NaN   NaN  NaN   NaN   NaN   
11     Regular season:  NaN  106.0  14.0     41.0  55.0 -2.0  58.0  60.0   
12          

,Season,Tournament/Team,№,GP,G,Assists,PTS,+/-,+,-,...,TIE/G,SFTE/G,TIPP/G,SFTPP/G,TISH/G,SFTSH/G,HITS,BLS,FOA,TkA
0,21/22 | playoffs,Jokerit (Helsinki),-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-,-,-,-,-,-,0.0,0.0,0.0,NaN
1,20/21 | playoffs,Jokerit (Helsinki),25,2.0,0.0,0.0,0.0,-1.0,0.0,1.0,...,12:37,17,01:08,1,00:00,0,4.0,0.0,1.0,NaN
2,20/21 | regular,Jokerit (Helsinki),25,50.0,8.0,18.0,26.0,-6.0,25.0,31.0,...,12:05,17.1,02:09,2.76,00:01,0.08,24.0,3.0,12.0,NaN
3,19/20 | playoffs,Jokerit (Helsinki),25,6.0,2.0,2.0,4.0,4.0,6.0,2.0,...,11:00,17,03:06,3.67,00:00,0,3.0,2.0,0.0,NaN
4,19/20 | regular,Jokerit (Helsinki),25,56.0,6.0,23.0,29.0,4.0,33.0,29.0,...,12:27,18.11,02:43,3.21,00:02,0.07,22.0,13.0,15.0,NaN
5,KHL Summary,Regular season:,NaN,106.0,14.0,41.0,55.0,-2.0,58.0,60.0,...,12:17,17.63,02:27,3,00:02,0.08,46.0,16.0,27.0,0.0
6,KHL Summary,Playoffs:,NaN,8.0,2.0,2.0,4.0,3.0,6.0,3.0,...,11:24,17,02:36,3,00:00,0,7.0,2.0,1.0,0.0
7,KHL Summary,KHL Total:,NaN,114.0,16.0,43.0,59.0,1.0,64.0,63.0,...,12:13,17.59,02:28,3,00:01,0.07,53.0,18.0,28.0,0.0


In [ ]:
def fix_player_games_in_khl_stat(player_games_in_khl_stat):
  player_games_in_khl_stat['Season'] = np.where(np.isnan(player_games_in_khl_stat['Assists']),player_games_in_khl_stat['Teams'], np.NaN)
  player_games_in_khl_stat['Season'] = player_games_in_khl_stat['Season'].fillna(method='ffill')
  player_games_in_khl_stat = player_games_in_khl_stat[player_games_in_khl_stat['Teams'] != player_games_in_khl_stat['Season']]
  player_games_in_khl_stat = player_games_in_khl_stat[['Season'] + [col for col in player_games_in_khl_stat.columns if col != 'Season']]
  player_games_in_khl_stat = player_games_in_khl_stat.reset_index(drop=True)
  return player_games_in_khl_stat
player_games_in_khl_stat = scrape_player_data('players/34721/')
print(player_khl_stat)
fix_player_games_in_khl_stat(player_games_in_khl_stat)

In [ ]:
def add_player_name(player_name, player_info,  player_khl_stat):


    player_info = pd.concat([player_name, player_info], axis=1)
    #КОСТЫЛЬ ИСПРАВИТЬ
    #player_info.columns=['URL',	'Player name',	'Born',	'Age',	'Country',	'Height',	'Weight', 'Shoots']
    #КОСТЫЛЬ ИСПРАВИТЬ


    player_khl_stat = pd.concat([player_name, player_khl_stat], axis=1)
    player_khl_stat[['URL','Player name']] = player_khl_stat[['URL','Player name']].fillna(method='ffill')
    player_khl_stat['Position'] = player_khl_stat['Position'].fillna(method='ffill')
   # else:

        # We do not need your kind in here.

       # pass

    return  player_info, player_khl_stat#, player_match_stat
#player_season_stat
player_khl_stat  = scrape_player_data('players/34721/')
#player_games_in_khl_stat = fix_player_games_in_khl_stat(player_games_in_khl_stat)
player_khl_stat = fix_player_khl_stat(player_khl_stat)
player_name, player_info = scrape_player_info('players/34721/')
#add_player_name(player_name, player_info, player_season_stat, player_match_stat)
player_info, player_khl_stat = add_player_name(player_name, player_info, player_khl_stat)
player_khl_stat

,URL,Player name,Position,Season,Tournament/Team,№,GP,G,Assists,PTS,...,TIE/G,SFTE/G,TIPP/G,SFTPP/G,TISH/G,SFTSH/G,HITS,BLS,FOA,TkA
0,https://en.khl.ru/players/34721/,Henrik Haapala,forward,21/22 | playoffs,Jokerit (Helsinki),-,0.0,0.0,0.0,0.0,...,-,-,-,-,-,-,0.0,0.0,0.0,NaN
1,https://en.khl.ru/players/34721/,Henrik Haapala,forward,20/21 | playoffs,Jokerit (Helsinki),25,2.0,0.0,0.0,0.0,...,12:37,17,01:08,1,00:00,0,4.0,0.0,1.0,NaN
2,https://en.khl.ru/players/34721/,Henrik Haapala,forward,20/21 | regular,Jokerit (Helsinki),25,50.0,8.0,18.0,26.0,...,12:05,17.1,02:09,2.76,00:01,0.08,24.0,3.0,12.0,NaN
3,https://en.khl.ru/players/34721/,Henrik Haapala,forward,19/20 | playoffs,Jokerit (Helsinki),25,6.0,2.0,2.0,4.0,...,11:00,17,03:06,3.67,00:00,0,3.0,2.0,0.0,NaN
4,https://en.khl.ru/players/34721/,Henrik Haapala,forward,19/20 | regular,Jokerit (Helsinki),25,56.0,6.0,23.0,29.0,...,12:27,18.11,02:43,3.21,00:02,0.07,22.0,13.0,15.0,NaN
5,https://en.khl.ru/players/34721/,Henrik Haapala,forward,KHL Summary,Regular season:,NaN,106.0,14.0,41.0,55.0,...,12:17,17.63,02:27,3,00:02,0.08,46.0,16.0,27.0,0.0
6,https://en.khl.ru/players/34721/,Henrik Haapala,forward,KHL Summary,Playoffs:,NaN,8.0,2.0,2.0,4.0,...,11:24,17,02:36,3,00:00,0,7.0,2.0,1.0,0.0
7,https://en.khl.ru/players/34721/,Henrik Haapala,forward,KHL Summary,KHL Total:,NaN,114.0,16.0,43.0,59.0,...,12:13,17.59,02:28,3,00:01,0.07,53.0,18.0,28.0,0.0


In [ ]:

list_profile_links = pd.DataFrame()
list_player_info = pd.DataFrame()
list_player_season_stat = pd.DataFrame()
list_player_khl_stat = pd.DataFrame()


alphabet = list(string.ascii_uppercase)
#alphabet = ['X']
for letter in alphabet:

    profile_links = get_profile_links(letter)
    print(profile_links)
    list_profile_links =  list_profile_links.append(profile_links, ignore_index=True)

# And let us save it straight away.

list_profile_links.to_csv('players_profile.csv', encoding='utf8', index=False)



for profile in list_profile_links['Profile link']:
    print(profile)
    player_name,player_info = scrape_player_info(profile)
    player_khl_stat = scrape_player_data(profile)

    player_khl_stat = fix_player_khl_stat(player_khl_stat)
    player_info, player_khl_stat = add_player_name(player_name, player_info, player_khl_stat)


    list_player_info = list_player_info.append(player_info, ignore_index=True)
   #list_player_season_stat = list_player_season_stat.append(player_season_stat, ignore_index=True)
    list_player_khl_stat = list_player_khl_stat.append(player_khl_stat, ignore_index=True)



list_player_career_stat = list_player_khl_stat[list_player_khl_stat['Season'] != 'KHL Summary']
list_player_season_stat = list_player_khl_stat[list_player_khl_stat['Season'] == 'KHL Summary']


list_player_info.to_csv('raw_players_info.csv', encoding='utf8', index=False)
list_player_career_stat.to_csv('raw_players_career.csv', encoding='utf8', index=False)
list_player_season_stat.to_csv('raw_players_khl_summary.csv', encoding='utf8', index=False)
list_player_khl_stat.to_csv('raw_players_khl_stat.csv', encoding='utf8', index=False)

       Profile link          Name Player                         Team  \
0   /players/16785/   Aaltonen Juhamatti           Jokerit (Helsinki)   
1   /players/17585/        Aaltonen Miro       Vityaz (Moscow Region)   
2   /players/13041/  Abdrakhmanov Ruslan            Amur (Khabarovsk)   
3   /players/38736/       Abdul Jindrich      Severstal (Cherepovets)   
5   /players/13873/       Abdullin Denis       Vityaz (Moscow Region)   
6   /players/19010/         Aberg Pontus        Traktor (Chelyabinsk)   
7   /players/16221/           Abid Ramzi        Traktor (Chelyabinsk)   
8   /players/19276/       Ablayev Kirill             Spartak (Moscow)   
9   /players/20861/        Abols Rodrigo                Dinamo (Riga)   
10  /players/23630/   Abramov Mikhail A.             HC Sochi (Sochi)   
11  /players/16673/    Abramov Sergei V.            Amur (Khabarovsk)   
12  /players/23248/       Abramov Vitaly                CSKA (Moscow)   
13  /players/17968/      Abrosimov Roman       Vity

<ipython-input-7-5e25f54b2bf4>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_profile_links =  list_profile_links.append(profile_links, ignore_index=True)


       Profile link          Name Player  \
0    /players/3950/        Babchuk Anton   
1   /players/21263/        Babenko Yegor   
2   /players/10892/      Babenko Yury I.   
3   /players/26798/       Babintsev Gleb   
4   /players/23764/     Babintsev Semyon   
5   /players/15943/       Bachurin Denis   
6   /players/23640/         Bacik Patrik   
8   /players/20553/    Backstrom Nicklas   
9   /players/21604/         Badun Alexei   
10   /players/4087/      Badyukov Alexei   
11  /players/22147/          Bailen Nick   
12  /players/33314/         Bailey Casey   
13  /players/39812/       Bakanov Andrei   
14  /players/14342/  Bakhriddinov Rustam   
15  /players/15063/      Bakika Vladimir   
16  /players/18926/         Bakos Martin   
17  /players/22190/      Bakushin Artyom   
18  /players/17678/      Balan Stanislav   
19   /players/4800/     Balandin Mikhail   
20  /players/18974/      Baldayev Viktor   
21  /players/23503/        Baldin Kirill   
22  /players/24048/       Balins

<ipython-input-7-5e25f54b2bf4>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_profile_links =  list_profile_links.append(profile_links, ignore_index=True)


       Profile link           Name Player                        Team  \
0   /players/15081/          Cajanek Petr      SKA (Saint Petersburg)   
1   /players/19530/      Cajkovsky Michal  Sibir (Novosibirsk Region)   
2   /players/17341/           Calder Kyle              Barys (Astana)   
3   /players/31274/          Calof Andrew           Amur (Khabarovsk)   
4   /players/41583/        Camara Anthony   Neftekhimik (Nizhnekamsk)   
5   /players/42355/        Campbell Colin   Kunlun Red Star (Beijing)   
6   /players/18937/         Carle Mathieu          Medvescak (Zagreb)   
7   /players/33476/          Caron Jordan  Sibir (Novosibirsk Region)   
9   /players/13203/          Caslava Petr     Severstal (Cherepovets)   
10  /players/24964/        Cehlarik Peter             Avangard (Omsk)   
11  /players/10857/           Cermak Leos   Torpedo (Nizhny Novgorod)   
12  /players/28628/           Cerny Tomas         Slovan (Bratislava)   
13  /players/16827/        Cervenka Roman      SKA 

...

<ipython-input-7-5e25f54b2bf4>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_info = list_player_info.append(player_info, ignore_index=True)
<ipython-input-7-5e25f54b2bf4>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_khl_stat = list_player_khl_stat.append(player_khl_stat, ignore_index=True)


/players/16125/


<ipython-input-7-5e25f54b2bf4>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_info = list_player_info.append(player_info, ignore_index=True)
<ipython-input-7-5e25f54b2bf4>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_khl_stat = list_player_khl_stat.append(player_khl_stat, ignore_index=True)


/players/9495/


<ipython-input-7-5e25f54b2bf4>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_info = list_player_info.append(player_info, ignore_index=True)
<ipython-input-7-5e25f54b2bf4>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_player_khl_stat = list_player_khl_stat.append(player_khl_stat, ignore_index=True)


In [ ]:
list_player_info

,URL,Player name,Position,"(Born,)","(Age,)","(Country,)","(Height,)","(Weight,)","(Shoots,)","(Contract to:,)","(Contract type:,)","(Died,)"
0,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,4 June 1985,37,Finland,184,89,right,NaN,NaN,NaN
1,https://en.khl.ru//players/17585/,Miro Aaltonen,forward,7 June 1993,29,Finland,177,84,left,NaN,NaN,NaN
2,https://en.khl.ru//players/13041/,Ruslan Abdrakhmanov,forward,25 December 1984,38,Russia,178,77,left,NaN,NaN,NaN
3,https://en.khl.ru//players/38736/,Jindrich Abdul,forward,1 February 1996,27,Czechia,185,85,left,NaN,NaN,NaN
4,https://en.khl.ru//players/13873/,Denis Abdullin,forward,1 January 1985,38,Russia,185,90,left,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
613,https://en.khl.ru//players/19061/,Viktor Zakharov,forward,8 January 1994,29,Ukraine,194,85,left,NaN,NaN,NaN
614,https://en.khl.ru//players/17610/,Vladilen Zakharov,forward,12 January 1994,29,Russia,194,104,right,NaN,NaN,NaN
615,https://en.khl.ru//players/33632/,Zakhar V. Zakharov,forward,9 January 2002,21,Russia,181,69,left,NaN,NaN,NaN
616,https://en.khl.ru//players/16125/,Alexander Zakirov,forward,8 July 1992,30,Russia,175,78,left,NaN,NaN,NaN


In [ ]:
list_player_career_stat

,URL,Player name,Position,Season,Tournament/Team,№,GP,G,Assists,PTS,...,TIE/G,SFTE/G,TIPP/G,SFTPP/G,TISH/G,SFTSH/G,HITS,BLS,FOA,TkA
0,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,15/16 | playoffs,Jokerit (Helsinki),50.0,6.0,0.0,1.0,1.0,...,11:25,17.33,03:55,4.0,00:00,0.0,3.0,0.0,1.0,NaN
1,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,15/16 | regular,Jokerit (Helsinki),50.0,45.0,15.0,13.0,28.0,...,12:21,17.8,02:35,3.33,00:08,0.22,10.0,10.0,9.0,NaN
2,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,14/15 | playoffs,Jokerit (Helsinki),50.0,10.0,4.0,2.0,6.0,...,12:01,18.2,02:51,4.0,00:00,0.1,4.0,1.0,3.0,NaN
3,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,14/15 | regular,Jokerit (Helsinki),50.0,48.0,10.0,24.0,34.0,...,12:26,16.56,03:14,3.92,00:02,0.06,14.0,6.0,9.0,NaN
4,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,11/12 | playoffs,Metallurg (Magnitogorsk),50.0,8.0,1.0,2.0,3.0,...,13:01,20.0,01:27,2.13,00:42,1.25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5202,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,11/12 | regular,Amur (Khabarovsk),10.0,33.0,2.0,10.0,12.0,...,10:55,16.0,00:22,1.39,01:50,3.12,NaN,NaN,NaN,NaN
5203,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,11/12 | regular,Summary:,NaN,41.0,2.0,11.0,13.0,...,10:40,15.51,00:34,1.61,01:29,2.51,NaN,NaN,NaN,NaN
5204,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,10/11 | regular,Amur (Khabarovsk),10.0,54.0,12.0,13.0,25.0,...,11:15,15.59,02:28,3.52,01:09,2.0,NaN,NaN,NaN,NaN
5205,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,09/10 | regular,Amur (Khabarovsk),10.0,42.0,3.0,3.0,6.0,...,09:38,12.79,00:42,1.43,01:18,2.14,NaN,NaN,NaN,NaN


In [ ]:
list_player_season_stat

,URL,Player name,Position,Season,Tournament/Team,№,GP,G,Assists,PTS,...,TIE/G,SFTE/G,TIPP/G,SFTPP/G,TISH/G,SFTSH/G,HITS,BLS,FOA,TkA
8,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,KHL Summary,Regular season:,NaN,201.0,67.0,65.0,132.0,...,12:21,17.57,02:49,3.72,00:23,0.81,24.0,16.0,18.0,0.0
9,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,KHL Summary,Playoffs:,NaN,44.0,11.0,8.0,19.0,...,13:12,19.95,02:19,3.14,00:10,0.45,7.0,1.0,4.0,0.0
10,https://en.khl.ru//players/16785/,Juhamatti Aaltonen,forward,KHL Summary,KHL Total:,NaN,245.0,78.0,73.0,151.0,...,12:30,18.0,02:44,3.62,00:21,0.74,31.0,17.0,22.0,0.0
21,https://en.khl.ru//players/17585/,Miro Aaltonen,forward,KHL Summary,Regular season:,NaN,244.0,69.0,106.0,175.0,...,13:34,20.49,03:19,3.83,00:45,1.55,175.0,111.0,56.0,0.0
22,https://en.khl.ru//players/17585/,Miro Aaltonen,forward,KHL Summary,Playoffs:,NaN,12.0,0.0,0.0,0.0,...,14:50,22.67,02:18,3.08,00:03,0.42,7.0,5.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,https://en.khl.ru//players/16125/,Alexander Zakirov,forward,KHL Summary,Regular season:,NaN,8.0,0.0,0.0,0.0,...,03:36,5.75,00:00,0.13,00:03,0.38,0.0,0.0,0.0,0.0
5198,https://en.khl.ru//players/16125/,Alexander Zakirov,forward,KHL Summary,KHL Total:,NaN,8.0,0.0,0.0,0.0,...,03:36,5.75,00:00,0.13,00:03,0.38,0.0,0.0,0.0,0.0
5207,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,KHL Summary,Regular season:,NaN,148.0,17.0,27.0,44.0,...,10:23,14.43,01:17,2.18,01:15,2.16,0.0,0.0,0.0,0.0
5208,https://en.khl.ru//players/9495/,Radik Zakiyev,forward,KHL Summary,Playoffs:,NaN,5.0,0.0,0.0,0.0,...,08:27,11.8,00:05,0.2,00:00,0.0,0.0,0.0,0.0,0.0
